In [41]:
from tensorflow.keras.datasets import cifar10
from matplotlib import pyplot as plt 
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [29]:
#Load cifar-10 dataset and one-hot encode target classes
def load_data():
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [30]:
# normalize input pixels
def normalize_pixels(x):
    x = x.astype('float32')
    return x / 255.0

In [31]:
# VGG inspired CNN
# Ref: https://arxiv.org/abs/1409.1556

# 4 VGG blocks
def create_cnn():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    # 10 classes
	model.add(Dense(10, activation='softmax'))
	# compile model
	model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
	return model

# 4 VGG blocks with increasing dropout, batch normalization, more dense layers
def create_cnn_improved():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
    # 10 classes
	model.add(Dense(10, activation='softmax'))
	# compile model
	model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
	return model

In [32]:
# plot diagnostic learning curves
# from https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
def summarize_diagnostics(result, model_name):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(result.result['loss'], color='blue', label='train')
	plt.plot(result.result['val_loss'], color='orange', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(result.result['accuracy'], color='blue', label='train')
	plt.plot(result.result['val_accuracy'], color='orange', label='test')
	# save plot to file
	plt.savefig(model_name + '_plot.png')
	plt.close()

In [35]:
def run_model_and_get_result(model, model_name):
    # Load and normalize data
    trainX, trainY, testX, testY = load_data()
    trainX, testX = normalize_pixels(trainX), normalize_pixels(testX)
    #Some data augmentation
    datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    it_train = datagen.flow(trainX, trainY, batch_size=64)
    # Fit model with augmented data and get results
    steps = int(trainX.shape[0] / 64)
    result = model.fit(it_train, steps_per_epoch=steps, epochs=50, validation_data=(testX, testY), verbose=0)
    # Evaluate results
    _, accuracy = model.evaluate(testX, testY, verbose=0)
    # Save the trained model
    model.save(model_name + '.h5')
    # Plot learning curve
    summarize_diagnostics(model_name, result)
    return result

In [1]:
initial = create_cnn()
acc = run_model_and_get_result(initial, 'initial')
print('Initial model accuracy: %.3f' % (acc * 100.0))
# > 75.310

NameError: name 'run_model_and_get_result' is not defined

In [ ]:
improved = create_cnn_improved()
acc_improved = run_model_and_get_result(improved, 'improved')
print('Improved model accuracy: %.3f' % (acc_improved * 100.0))

In [6]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

ModuleNotFoundError: No module named 'keras'